### Include Dyn3d and config files

In [ ]:
include(Pkg.dir("Dyn3d")*"/src/Dyn3d.jl")
using Dyn3d

include(joinpath(Pkg.dir("Whirl"), "src/Whirl.jl"))
using Whirl

using Reactive, Interact
using Plots
pyplot()

### Choose from configuration files

In [ ]:
include("../src/config_files/2dFallingCircle.jl")

### Build joint-body chain

In [ ]:
bodys, joints, system = BuildChain(config_bodys, config_joints, 
                                   config_system)
bd = BodyDyn(bodys, joints, system)

### Initialize system state

In [ ]:
# init system
bd, soln = InitSystem!(bd)

# init soln structure
solns = (Soln)[]
push!(solns, soln)

# init VertsHistory struct
vs = []
push!(vs, VertsHistory(system.nbody, bd.bs))

nothing

### Set up HERKBody object

In [ ]:
herk = HERKBody(system.num_params,HERKFuncM, HERKFuncGT, HERKFuncG,
                (HERKFuncf,HERKFuncgti), (UpdatePosition!,UpdateVelocity!))

### Time Marching

In [ ]:
# advance in time
idx = 0
@time begin
while soln.t < tf
    # advance one step
    soln, bd.bs, bd.js, bd.sys = herk(soln, bd, _isfixedstep=true)
        
    # record soln and verts_i info
    push!(solns, soln)
    push!(vs, VertsHistory(system.nbody, bd.bs))
        
    # print progress
    idx += 1
    if mod(idx,500) == 1 
        @printf("itr = %d, t = %.3f, dt = %e\n", idx, soln.t, soln.dt)
#         println("center of mass at", MassCenter(bodys, system))
    end
end
@printf("itr = %d, t = %.3f, dt = %e\n", idx, soln.t, soln.dt)
end

### Save verts data on regulated time grid for movie in Matlab

In [ ]:
bodys, joints, system = bd.bs, bd.js, bd.sys

using MAT
using Interpolations

# create regular time grid and acquire solutions on it
qJ_regs = Float64[]
t_reg = linspace(0,solns[end].t,length(solns))
for i = 1:system.ndof
    t_temp = ([solns[k].t for k = 1:length(solns)],)
    qJ_temp = [solns[k].qJ[i] for k = 1:length(solns)]   
    qJ_reg = interpolate(t_temp, qJ_temp, Gridded(Linear()))[t_reg]
    append!(qJ_regs, qJ_reg)
end
qJ_regs = reshape(qJ_regs,(length(solns), system.ndof))

# get verts info based on this regular grid solution
vs_reg = []
bodys_reg = deepcopy(bodys)
joints_reg = deepcopy(joints)
system_reg = deepcopy(system)
for i = 1:length(solns)
    bodys_reg, joints_reg, system_reg = UpdatePosition!(bodys_reg, joints_reg, system_reg, solns[i].qJ)
    push!(vs_reg, VertsHistory(system.nbody, bodys_reg))
end

# write to .mat file for animation
# vs_reg follow the form [ntime][nbody, nverts, xyz]
matwrite("../matlab_plot/verts_i.mat", Dict(
    "ndim" => system.ndim,
    "nbody" => system.nbody,
    "nverts" => bodys[1].nverts,
    "t" => collect(t_reg),
    "verts" => vs_reg
))

### Interactive plot

In [ ]:
freq = 100 # intervals of plotting
camera_view = (45,45)
v = isdefined(:plot_dir) ? plot_dir : system.ndim==2 ? (1,2) : (1,2,3)

@manipulate for i in 1:Int(round(length(t_reg)/freq)-1)

    if system.ndim == 2
        k = i*freq+1
        # first body
        p = plot(vs_reg[k][1,:,v[1]],vs_reg[k][1,:,v[2]], linewidth=3, aspect_ratio=:equal)        
        # the rest bodys
        for j in 2:system.nbody
            plot!(vs_reg[k][j,:,v[1]],vs_reg[k][j,:,v[2]], linewidth=3, aspect_ratio=:equal)            
        end    
        plot!(xlims=(-2,2), ylims=(-2,2))
        
    else
        k = i*freq+1
        h = [1:bodys[1].nverts;1]
        # first body        
        plot(vs_reg[k][1,h,v[1]],vs_reg[k][1,h,v[2]],vs_reg[k][1,h,v[3]], linewidth=3, aspect_ratio=:equal ,camera = camera_view)
        # the rest bodys
        for j in 2:system.nbody
            plot!(vs_reg[k][j,h,v[1]],vs_reg[k][j,h,v[2]],vs_reg[k][j,h,v[3]], linewidth=3, aspect_ratio=:equal ,camera = camera_view)
        end    
        plot!(xlims=(-2,2), ylims=(-2,2), zlims=(-2,2))        
    end

end